In [ ]:
#importing required libraries

import requests
import os
import pandas as pd
pd.set_option('display.max_columns', None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
os.makedirs("/content/drive/MyDrive/Thesis/Training_Data/")

In [ ]:
os.chdir("/content/drive/MyDrive/Thesis/Training_Data/")

In [ ]:
os.getcwd()

'/content/drive/MyDrive/Thesis/Training_Data'

In [ ]:
biomedical_images = ['X-ray', 'Magnetic Resonance Imaging (MRI)', 'Computed Tomography (CT) Scan', 'Positron Emission Tomography (PET) Scan', 'Ultrasound', 'Endoscopy', 'Histology']

In [ ]:
# Openi API endpoint
openi_endpoint = "https://openi.nlm.nih.gov/api/search"

# search term
#query = "mri"
initial_value = 1

# for number of loops enter m
number_of_loops = 20
# for a loop of value n enter n-1
loop = 99

In [ ]:
# Initialize list to hold downloaded image URLs
downloaded_image_urls = []
downloaded_pmc_urls = []
image_captions = []
image_class = []

def download_images(initial_value, number_of_loops, loop, openi_endpoint, image_type):
  # loop for n times
  for num in range(number_of_loops):
      # Send the request with updated parameters
      params = {
          "query": image_type,
          "it": "i",
          "m": initial_value,
          "n": initial_value + int(loop),
          "outputFormat": "JSON"
      }

      # Create folder to save images if it doesn't exist
      folder_name = image_type
      if not os.path.exists(folder_name):
          os.makedirs(folder_name)
      # Send the Openi API search request
      response = requests.get(openi_endpoint, params=params)

      # Do something with the response data here
      #print(response.json())
      # Check if the request was successful
      if response.status_code == 200:

          # Parse the JSON response and get the total number of results
          response_data = response.json()
          #num_results = 10
          num_results = response_data["total"]

          # Download the remaining images in batches of 100
          print(f"Downloading images {initial_value} to {initial_value+loop}...")
          response = requests.get("https://openi.nlm.nih.gov/api/search", params=params)
          if response.status_code == 200:
              response_data = response.json()

              for i, result in enumerate(response_data["list"]):
                  article_UID = result["uid"]
                  # Get the image URL and add the missing scheme if necessary
                  image_url = result["imgLarge"]
                  if image_url.startswith("/"):
                      image_url = "https://openi.nlm.nih.gov" + image_url

                  #print(image_url)
                  downloaded_image_urls.append(image_url)
                  pmc_urls = result.get("pmc_url", "")
                  #pmc_urls = result["pmc_url"]
                  #print(pmc_urls)
                  downloaded_pmc_urls.append(pmc_urls)
                  caption_of_image = result["image"]["caption"]
                  image_captions.append(caption_of_image)
                  image_class.append(image_type)
                  #print(i)
                  image_data = requests.get(image_url).content
                  # Get the extension and append the characters before it to the filename
                  characters_before_extension = image_url.split("/")[-1][0:-4]
                  #print(characters_before_extension)
                  # Save the image data to a file
                  filename = f"{folder_name}/{characters_before_extension}.jpg"
                  with open(filename, "wb") as f:
                      f.write(image_data)
                  #filename = f"{folder_name}/image_{offset+i}.jpg"
                  #with open(filename, "wb") as f:
                  #    f.write(image_data)
                  print(f"Image {initial_value+i}/{num_results} saved to {filename}")
          else:
              print(f"Openi API search failed with status code {response.status_code}")
      else:
          print(f"Openi API search failed with status code {response.status_code}")

      # Increase the initial value by loop + 1
      initial_value += (loop + 1)

  # Print the list of downloaded image URLs
  #print("Downloaded image URLs:")
  #print(downloaded_image_urls)

  # Print the list of downloaded image URLs
  #print("Downloaded image article URLs:")
  #print(downloaded_pmc_urls)


In [ ]:
for index, type in enumerate(biomedical_images):
  download_images(initial_value, number_of_loops, loop, openi_endpoint, image_type=biomedical_images[index])

In [ ]:
print(len(downloaded_image_urls))
print(len(downloaded_pmc_urls))
print(len(image_captions))

14000
14000
14000


In [ ]:
info_data = pd.DataFrame()
info_data['image_urls'] = downloaded_image_urls
info_data['article_urls'] = downloaded_pmc_urls
info_data['image_captions'] = image_captions
info_data['image_class'] = image_class

In [ ]:
info_data.shape

(14000, 4)

In [ ]:
info_data.head(5)

,image_urls,article_urls,image_captions,image_class
0,https://openi.nlm.nih.gov/imgs/512/42/5137654/...,http://www.ncbi.nlm.nih.gov/pmc/articles/PMC51...,Images of a 12-year-old girl who underwent hem...,X-ray
1,https://openi.nlm.nih.gov/imgs/512/392/3121776...,http://www.ncbi.nlm.nih.gov/pmc/articles/PMC31...,Experimental setup and new 3D-SGXM system.(A) ...,X-ray
2,https://openi.nlm.nih.gov/imgs/512/244/3853854...,http://www.ncbi.nlm.nih.gov/pmc/articles/PMC38...,(a) Rib fusion on <b>X-ray</b> chest (white ar...,X-ray
3,https://openi.nlm.nih.gov/imgs/512/136/3343273...,http://www.ncbi.nlm.nih.gov/pmc/articles/PMC33...,Correlative live-cell pre-scanning and in-colu...,X-ray
4,https://openi.nlm.nih.gov/imgs/512/323/5334024...,http://www.ncbi.nlm.nih.gov/pmc/articles/PMC53...,Sequential plain anteroposterior <b>radiograph...,X-ray
5,https://openi.nlm.nih.gov/imgs/512/179/2705726...,http://www.ncbi.nlm.nih.gov/pmc/articles/PMC27...,A 45-year-old man with a personal and familial...,X-ray
6,https://openi.nlm.nih.gov/imgs/512/173/4497388...,http://www.ncbi.nlm.nih.gov/pmc/articles/PMC44...,Induction of DNA double-strand breaks by soft ...,X-ray
7,https://openi.nlm.nih.gov/imgs/512/248/3709498...,http://www.ncbi.nlm.nih.gov/pmc/articles/PMC37...,Results of the numerical simulations of <b>X-r...,X-ray
8,https://openi.nlm.nih.gov/imgs/512/136/2678014...,http://www.ncbi.nlm.nih.gov/pmc/articles/PMC26...,Debye scattering curves calculated for the mod...,X-ray
9,https://openi.nlm.nih.gov/imgs/512/4/4648396/P...,http://www.ncbi.nlm.nih.gov/pmc/articles/PMC46...,Advantages of phase contrast <b>X-ray</b> imag...,X-ray


In [ ]:
info_data.to_csv(f"description.csv",index=False)

In [ ]:
# checking the amount of images for the classes
for num in range(len(biomedical_images)):
    print(biomedical_images[num])
    # Send the request with updated parameters
    params = {
        "query": biomedical_images[num],
        "outputFormat": "JSON"
    }

    # Send the Openi API search request
    response = requests.get(openi_endpoint, params=params)

    # Check if the request was successful
    if response.status_code == 200:

        # Parse the JSON response and get the total number of results
        response_data = response.json()
        num_results = response_data["total"]
        print(num_results)

X-ray
274894
Magnetic Resonance Imaging (MRI)
149118
Computed Tomography (CT) Scan
75425
Positron Emission Tomography (PET) Scan
13271
Ultrasound
115594
Endoscopy
43042
Histology
266674
